**Team Members**

Avneesh Nolkha

Ayushee Mittal

Smita Sasindran

Vishesh Sethi



## **Importing Libraries**

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## **Transforms**

In [2]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                      transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

## **Dataset**

In [3]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

## **Dataloader**

In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


## **Ghost Batch Normalization classes**

In [5]:
class BatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, weight=True, bias=True):
        super().__init__(num_features, eps=eps, momentum=momentum)
        self.weight.data.fill_(1.0)
        self.bias.data.fill_(0.0)
        self.weight.requires_grad = weight
        self.bias.requires_grad = bias


class GhostBatchNorm(BatchNorm):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        self.register_buffer('running_mean', torch.zeros(num_features * self.num_splits))
        self.register_buffer('running_var', torch.ones(num_features * self.num_splits))

    def train(self, mode=True):
        if (self.training is True) and (mode is False):  # lazily collate stats when we are going to use them
            self.running_mean = torch.mean(self.running_mean.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
            self.running_var = torch.mean(self.running_var.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
        return super().train(mode)

    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            return F.batch_norm(
                input.view(-1, C * self.num_splits, H, W), self.running_mean, self.running_var,
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
        else:
            return F.batch_norm(
                input, self.running_mean[:self.num_features], self.running_var[:self.num_features],
                self.weight, self.bias, False, self.momentum, self.eps)



## **Model**

In [6]:
dropout_value = 0.09
class Net(nn.Module):
    def __init__(self, gbn, gbn_split):
        super(Net, self).__init__()

        self.gbn = gbn
        self.gbn_split = gbn_split

        print("Is GBN run:", gbn)
        # Add BatchNorm or GhostBatchNorm
        def BNLayer(n_features):
            bn = nn.BatchNorm2d(n_features) if not self.gbn else GhostBatchNorm(n_features, self.gbn_split) 
            return bn


        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            #nn.BatchNorm2d(10),
            BNLayer(10),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            #nn.BatchNorm2d(16),
            BNLayer(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU(),
            #nn.BatchNorm2d(10)
            nn.BatchNorm2d(10)
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            #nn.BatchNorm2d(10),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            #nn.BatchNorm2d(12),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            #nn.BatchNorm2d(16),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=20, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            #nn.BatchNorm2d(20),
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        ) 

        self.dropout = nn.Dropout(dropout_value)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)        
        x = self.convblock8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

## **Model Params**

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net(False, 1).to(device)
summary(model, input_size=(1, 28, 28))

cuda
Is GBN run: False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,440
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
             ReLU-10           [-1, 10, 24, 24]               0
      BatchNorm2d-11           [-1, 10, 24, 24]              20
        MaxPool2d-12           [-1, 10, 12, 12]               0
           Conv2d-13           [-1, 10, 10, 10]             900
             ReL

## **Setting up parameters for 5 runs**

In [8]:
run_params = [
  {'model': 'L1_with_BN', 'l1': True, 'l2': False, 'gbn': False, 'gbn_split': 1, 'l_l1': 0.0000085},
  {'model': 'L2_with_BN', 'l1': False, 'l2': True, 'gbn': False, 'gbn_split': 1, 'l_l1': 0},
  {'model': 'L1_and_L2_with_BN', 'l1': True, 'l2': True, 'gbn': False, 'gbn_split': 1, 'l_l1': 0.00000000009},
  {'model': 'GBN', 'l1': False, 'l2': False, 'gbn': True, 'gbn_split': 2, 'l_l1': 0},
  {'model': 'L1_and_L2_with_GBN', 'l1': True, 'l2': True, 'gbn': True, 'gbn_split': 2, 'l_l1': 0.00000000000001}
]
    

In [9]:

train_losses = {}
train_acc = {}
test_losses = {}
test_acc = {}
for rp in run_params:
  train_losses[rp['model']] = []
  train_acc[rp['model']] = []
  test_losses[rp['model']] = []
  test_acc[rp['model']] = []



## **Training and Testing Loops**

In [10]:
from tqdm import tqdm

# train_losses = []
# train_acc = []
# test_losses = []
# test_acc = []

misclassified = []

#{'model': 'L1_with_BN', 'l1': True, 'l2': False, 'gbn': False, 'gbn_split': 1},
def train(model, device, train_loader, optimizer, epoch, params):
  name = params['model']

  model.train()
  pbar = tqdm(train_loader, position=0)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    l1_reg = 0
    l2_reg = 0

    if params['l1']:
      for param in model.parameters():
        l1_reg += param.abs().sum()

    # if params['l2']:
    #   for param in model.parameters():
    #     l2_reg += param.norm(2)

    # Calculate loss
    loss = F.nll_loss(y_pred, target) + params['l_l1'] * l1_reg #+ lambda_l2 * l2_reg

    #train_losses.append(loss)
    train_losses[name].append(loss)


    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    #train_acc.append(100*correct/processed)
    train_acc[name].append(100*correct/processed)
  

def test(model, device, test_loader, params):
    name = params['model']

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #test_losses.append(test_loss)
    test_losses[name].append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    #test_acc.append(100. * correct / len(test_loader.dataset))
    test_acc[name].append(100. * correct / len(test_loader.dataset))

    

## **Running the models**

In [ ]:
models = {}

optim_params = {
  'L1_with_BN':        {'lr':0.025, 'step_size': 12,'gamma': 0.2, 'weight_decay': 0},
  'L2_with_BN':        {'lr':0.025, 'step_size': 6, 'gamma': 0.1, 'weight_decay': 0.000095},
  'L1_and_L2_with_BN': {'lr':0.025, 'step_size': 6, 'gamma': 0.1, 'weight_decay': 0.000009},
  'GBN':               {'lr':0.025, 'step_size': 6, 'gamma': 0.1, 'weight_decay': 0},
  'L1_and_L2_with_GBN':{'lr':0.025, 'step_size': 6, 'gamma': 0.1, 'weight_decay': 0.0000001} 
}


for i, rp in enumerate(run_params):
    name = rp['model']
    opt = optim_params[name]
    print("\n\n ******** Running for: ", name, "====================================================")
    model =  Net(rp['gbn'], rp['gbn_split']).to(device)
    
    #optimizer = optim.SGD(model.parameters(), lr=0.0180, momentum=0.92)
    optimizer = optim.SGD(model.parameters(), lr=opt['lr'], momentum=0.9, weight_decay=opt['weight_decay'])
    scheduler = StepLR(optimizer, step_size=opt['step_size'], gamma=opt['gamma'])

    models[name] = model

    EPOCHS = 25
    for epoch in range(EPOCHS):
        print("EPOCH:", epoch+1)
        train(model, device, train_loader, optimizer, epoch, rp)
        scheduler.step()
        test(model, device, test_loader, rp)


  0%|          | 0/469 [00:00<?, ?it/s]



 ******** Running for:  L1_with_BN ====================================================
Is GBN run: False
EPOCH: 1


Loss=0.16970933973789215 Batch_id=468 Accuracy=90.74: 100%|██████████| 469/469 [00:13<00:00, 33.81it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0586, Accuracy: 9832/10000 (98.32%)

EPOCH: 2


Loss=0.09658050537109375 Batch_id=468 Accuracy=97.79: 100%|██████████| 469/469 [00:14<00:00, 32.92it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...

Loss=0.05114399269223213 Batch_id=468 Accuracy=98.88: 100%|██████████| 469/469 [00:13<00:00, 35.55it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0194, Accuracy: 9935/10000 (99.35%)

EPOCH: 11


Loss=0.00981215387582779 Batch_id=468 Accuracy=98.87: 100%|██████████| 469/469 [00:13<00:00, 35.71it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0194, Accuracy: 9941/10000 (99.41%)

EPOCH: 12


Loss=0.01965942233800888 Batch_id=175 Accuracy=98.92:  37%|███▋      | 175/469 [00:04<00:08, 36.50it/s]

## **Plotting functions**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from google.colab import files

def plot_with_legends(run_params, data, title):
    figure = plt.figure()
    #figure.suptitle(title, fontsize=16)
    plt.figure(figsize=(15,8))
    
    for i, pld in enumerate(run_params):
        name = pld['model']
        plt.plot(data[name], label=name)

    plt.legend()
    plt.title(title)
    fname = name + "_" + title  
    plt.savefig(fname + ".png")
    files.download(fname + ".png") 
    plt.show()  


In [ ]:
plot_with_legends(run_params, train_acc, 'Training Accuracy')


In [ ]:

plot_with_legends(run_params, test_acc, 'Test Accuracy')


In [ ]:
plot_with_legends(run_params, train_losses, 'Training Loss')


In [ ]:
plot_with_legends(run_params, test_losses, 'Test Loss')

## **Misclassified Images with GBN**

In [ ]:
gbn_model = models['GBN']


incorrect_indexes = {} # {23: {'actual': 1, 'predicted': 4}}
gbn_model.eval()
count = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = gbn_model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

        for sampleno in range(data.shape[0]):
                if(target[sampleno]!=pred[sampleno]):
                    count += 1
                    #print("Index=", sampleno, ", Actual=", target[sampleno].cpu().numpy(), ", Predicted: ", pred[sampleno].cpu().numpy()[0])
                    incorrect_indexes[sampleno] = {'actual': target[sampleno].cpu().numpy(), 
                                                   'predicted': pred[sampleno].cpu().numpy()[0], 
                                                   'data': data[sampleno].cpu().numpy()}

#print(incorrect_indexes) 
print(len(incorrect_indexes))
print(count)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

#{23: {'actual': 1, 'predicted': 4}}
x = 0
y = 0
fig, axs = plt.subplots(5, 5, figsize = (15, 15))
plt.setp(axs, xticks=[], yticks=[])
fig.subplots_adjust(wspace=0.7)
images = list(incorrect_indexes.items())[:25]
for index, results in images:
  #print(index)
  img = results['data'] 
  img = np.squeeze(img)
  actual_class = results['actual'] 
  predicted_class =  results['predicted'] 

  plt.savefig("misclassified.png")
  files.download("misclassified.png")  
  axs[x, y].imshow(img)
  axs[x, y].set_title('Actual Class:' + str(actual_class) + "\nPredicted class: " + str(predicted_class))
  
  if y == 4:
    x += 1
    y = 0
  else:
    y += 1

plt.savefig("misclassified.png")
files.download("misclassified.png")     